In [1]:
from bs4 import BeautifulSoup
import re
import requests
#from urllib.request import urlopen as uReq, Request

import pandas as pd
import numpy as np

import spacy
import spacy.cli

In [2]:
from time import sleep
from random import randint

In [3]:
dic_sentimentos = {}
d = 4

sentimentos_originais = ['alegria','medo','nojo','raiva','tristeza']
lista_sent = [('alegria','alegria', 0), ('medo','medo', 0), ('nojo','nojo', 0), ('raiva','raiva', 0), ('tristeza','tristeza', 0)]
tam_lista_sent = 5

In [4]:
def busca_inicial():
    global lista_sent
    global tam_lista_sent
    
    for i in range(0, tam_lista_sent):
        s, o, h = lista_sent.pop(0)
        dic_sentimentos[s] = o
        url = "https://www.sinonimos.com.br/" + s
        soup = BeautifulSoup(requests.get(url).text, "html5lib")
        retorno = soup.find_all('a', class_ ='sinonimo')
        lista_sent = lista_sent + list(pd.Series(retorno).apply(lambda x: (str(x).split('>')[1][:-3], o, h+1)))
        
        sleep(randint(1,3))
    

In [5]:
def verifica_balanco():    
    qt_dicionario = pd.DataFrame(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()).sort_index()['sent'].tolist()
    qt_dicionario = np.array(qt_dicionario)
    
    menor_valor = qt_dicionario.min()
    
    if (qt_dicionario-menor_valor).min() > 1:
        return 0
    else:
        return 1    

In [6]:
def busca_indice_elemento(sentimento):
    global lista_sent
    global tam_lista_sent
    
    sent_origem = 'nulo'    
    i = 0
    
    while i < len(lista_sent) and sent_origem != sentimento:
        s, sent_origem, h = lista_sent[i]
        i += 1
        
    if i == len(lista_sent):
        return -1
    else:
        return i-1   

In [7]:
def dicionario_por_grupo():
    global lista_sent
    global tam_lista_sent
    
    aux_lista_sent = []
    aux_dic_sentimentos = {}
    
    for i in range(0, tam_lista_sent):
        indice = busca_indice_elemento(str(sentimentos_originais[i]))
        
        if indice != -1:
            s, o, h = lista_sent.pop(indice)
            
            while s in dic_sentimentos or s in aux_dic_sentimentos:
                indice = busca_indice_elemento(sentimentos_originais[i])
                if indice != -1:
                    s, o, h = lista_sent.pop(indice)
                else:
                    return aux_lista_sent, {}
            
            if indice != -1:
                aux_dic_sentimentos[s] =  o    
                url = "https://www.sinonimos.com.br/" + s
                soup = BeautifulSoup(requests.get(url).text, "html5lib")            
                retorno = soup.find_all('a', class_ ='sinonimo')
                aux_lista_sent = aux_lista_sent + list(pd.Series(retorno).apply(lambda x: (str(x).split('>')[1][:-3], o, h+1)))
            else:
                return aux_lista_sent, {}
        else:
            return aux_lista_sent, {}
        
    return aux_lista_sent, aux_dic_sentimentos

In [8]:
def cria_dicionario():
    global lista_sent
    global d
    
    for i in range(0, d):       
    
        lista_sent_provisoria = []
        aux_dic = {'a': 0}

        while lista_sent and aux_dic: 
            aux_lista, aux_dic = dicionario_por_grupo()
            lista_sent_provisoria = lista_sent_provisoria + aux_lista    
            dic_sentimentos.update(aux_dic)  
            
        lista_sent = lista_sent_provisoria[:]

In [9]:
busca_inicial()
cria_dicionario()

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [10]:
pd.DataFrame(pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()).sort_index()

,sent
alegria,3072
medo,3072
nojo,3072
raiva,3072
tristeza,3072


In [11]:
(pd.DataFrame.from_dict(dic_sentimentos, orient='index')).to_csv('dic_sentimento_4_niveis_balanceado.txt', sep='\t' )